mini-ImageNet 데이터셋은 100개의 클래스가 있고 클래스당 84*84 색상 이미지가 60개가 있다.

In [79]:
!pip install pandas
!pip install matplotlib

In [80]:
import pandas as pd
train_csv = pd.read_csv('C:\\Users\\OWNER\\github\\ReinforcementLearning\\MetalearningClasifier\\Mini_ImageNet_csv\\train.csv')
train_csv

,filename,label
0,n0153282900000005.jpg,n01532829
1,n0153282900000006.jpg,n01532829
2,n0153282900000007.jpg,n01532829
3,n0153282900000010.jpg,n01532829
4,n0153282900000014.jpg,n01532829
...,...,...
38395,n1313361300001288.jpg,n13133613
38396,n1313361300001290.jpg,n13133613
38397,n1313361300001296.jpg,n13133613
38398,n1313361300001297.jpg,n13133613


In [81]:
labels = train_csv['label'].unique()
new_labels = dict(zip(labels, range(len(labels))))

train_csv['label'] = train_csv['label'].replace(new_labels)
train_csv

C:\Users\OWNER\AppData\Local\Temp\ipykernel_3856\25485979.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_csv['label'] = train_csv['label'].replace(new_labels)


,filename,label
0,n0153282900000005.jpg,0
1,n0153282900000006.jpg,0
2,n0153282900000007.jpg,0
3,n0153282900000010.jpg,0
4,n0153282900000014.jpg,0
...,...,...
38395,n1313361300001288.jpg,63
38396,n1313361300001290.jpg,63
38397,n1313361300001296.jpg,63
38398,n1313361300001297.jpg,63


In [89]:
import torch
import learn2learn as l2l
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms

# 1. 데이터셋 로드
# Mini-ImageNet 데이터셋 로드 (5-way 1-shot 설정)
train_dataset = l2l.vision.datasets.MiniImagenet(
    root='~/data',
    mode='train',
    download=True,
    transform=transforms.Compose([
        transforms.Resize((84, 84)),
        transforms.ToTensor(),
    ])
)

# 데이터셋 샘플링 (5-way 1-shot)
train_tasks = l2l.data.TaskDataset(
    train_dataset,
    task_transforms=[
        l2l.data.transforms.NWays(train_dataset, n=5),  # 5-way
        l2l.data.transforms.KShots(train_dataset, k=1),  # 1-shot
        l2l.data.transforms.LoadData(train_dataset),
    ],
    num_tasks=20000,  # 생성할 작업(Task)의 수
)

# 2. 모델 정의
class SimpleCNN(nn.Module):
    def __init__(self, output_size):
        super(SimpleCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, 3),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.classifier = nn.Linear(64 * 19 * 19, output_size)

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        return self.classifier(x)

model = SimpleCNN(output_size=5)  # 5-way 분류

# 3. MAML 알고리즘 초기화
maml = l2l.algorithms.MAML(model, lr=0.01)  # Inner loop 학습률

# 4. 메타 옵티마이저 정의
meta_optimizer = optim.Adam(maml.parameters(), lr=0.001)  # Outer loop 학습률

# 5. 메타 학습 루프
for iteration in range(1000):  # 메타 학습 반복
    meta_optimizer.zero_grad()  # 메타 옵티마이저 초기화
    task_loss = 0.0

    # 각 작업(Task)에서 학습
    for task in range(5):  # 5개의 작업(Task) 샘플링
        learner = maml.clone()  # 모델 복제 (Inner loop용)
        batch = train_tasks.sample()  # 작업(Task) 샘플링
        data, labels = batch

        # Inner loop: 지원 집합(Support Set) 학습
        adaptation_data, adaptation_labels = data[:5], labels[:5]  # 5-way 1-shot
        adaptation_loss = nn.CrossEntropyLoss()(learner(adaptation_data), adaptation_labels)
        learner.adapt(adaptation_loss)  # Inner loop 업데이트

        # Outer loop: 질의 집합(Query Set) 평가
        evaluation_data, evaluation_labels = data[5:], labels[5:]  # 나머지 데이터
        evaluation_loss = nn.CrossEntropyLoss()(learner(evaluation_data), evaluation_labels)
        task_loss += evaluation_loss

    # 메타 그래디언트 계산 및 업데이트
    task_loss /= 5  # 평균 손실
    task_loss.backward()  # 그래디언트 계산
    meta_optimizer.step()  # 메타 옵티마이저 업데이트

    # 로그 출력
    if iteration % 100 == 0:
        print(f"Iteration {iteration}, Meta Loss: {task_loss.item()}")

ModuleNotFoundError: No module named 'learn2learn'

In [93]:
!pip install -U learn2learn

  Using cached learn2learn-0.2.0.tar.gz (7.0 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached gym-0.26.2.tar.gz (721 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached scipy-1.15.2-cp313-cp313-win_amd64.whl.metadata (60 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Usin

  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [19 lines of output]
      + c:\Users\OWNER\github\ReinforcementLearning\.venv\Scripts\python.exe C:\Users\OWNER\AppData\Local\Temp\pip-install-z1s0vr4a\numpy_96474838fbd14ed491248fdfd8df711c\vendored-meson\meson\meson.py setup C:\Users\OWNER\AppData\Local\Temp\pip-install-z1s0vr4a\numpy_96474838fbd14ed491248fdfd8df711c C:\Users\OWNER\AppData\Local\Temp\pip-install-z1s0vr4a\numpy_96474838fbd14ed491248fdfd8df711c\.mesonpy-uaumyz8v -Dbuildtype=release -Db_ndebug=if-release -Db_vscrt=md --native-file=C:\Users\OWNER\AppData\Local\Temp\pip-install-z1s0vr4a\numpy_96474838fbd14ed491248fdfd8df711c\.mesonpy-uaumyz8v\meson-python-native-file.ini
      The Meson build system
      Version: 1.2.99
      Source dir: C:\Users\OWNER\AppData\Local\Temp\pip-install-z1s0vr4a\numpy_96474838fbd14ed491248fdfd8df711c
      Build dir: C:\Users\OWNER\AppData\Local\Temp\pip-install-